We will be modifying ReAct a little to take advantage of the progress that has been made in LLMs since the paper was originally released. We make two modifications:

1. We use chat models, when ReAct was released LLMs were not fine-tuned specifically for chat and instead were prompted to generate chat-like dialogues. Most SotA LLMs nowadays are built specifically for chat and so the input into them must be modified to be chat-model friendly.

2. We will use JSON-mode to force structured output from our LLMs. The original ReAct method simply instructed the LLM to output everything in a particular format. That works but is prone to occasionally breaking. By forcing JSON-like output we reduce the likelihood of poorly structured output. To accomodate this we modify the instructions to ask for `thought` and `action` steps in a JSON format.

In [1]:
system_prompt = """
You are a helpful assistant. Given a user query you must provide a `thought` and
`action` step that take one step towards solving the user's query. Both the
`thought` and `action` steps will be contained in JSON output.

The `thought` is a JSON object with a `thoughts` key containing your thoughts
on how to solve the user's query.

The `action` step is a JSON object with a `tool` key containing the name of the
tool to use and a `args` key containing a JSON object of arguments to pass to
the tool.

Here is an example:

user: What is the weather in Tokyo?

```json
{
  "thought": {"thoughts": "I need to find out the current temperature in Tokyo"},
  "action": {"tool": "search", "args": {"query": "current temperature in Tokyo"}}
}
```

If you have performed any previous thought and action steps, you will find them
below under the "Previous Steps" section. Alongside these you will find an
`observation` key containing the output of those previous actions.
"""

We haven't defined any tools or agent logic yet, but let's see what type of output
our LLM produces if we prompt it with this system prompt.

Make sure you have Llama 3.2 downloaded already via:

```
ollama run llama3.2:3b-instruct-fp16
```

In [2]:
import ollama

res = ollama.chat(
    model="llama3.2:3b-instruct-fp16",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "What is the date today?"},
    ],
    format="json",
)

print(res["message"]["content"])

{
  "thought": {
    "thoughts": "I need to find out the current date"
  },
  "action": {
    "tool": "date",
    "args": {}
  }
}
  
  
  
 


Great, we're outputting the correct format *however* we don't have a `date` tool, in fact, we don't have *any* tools! Let's define some.

First, we'll define a search tool. This tool will allow our agent to search the web for information. To implement it we will use the Tavily API, it comes with a number of requests for free but we do need to [sign up for the API](https://app.tavily.com/home) and get an API key to use it.

In [5]:
import requests

TAVILY_API_KEY = "tvly-..."

tavily_url = "https://api.tavily.com"

res = requests.post(
    f"{tavily_url}/search",
    json={
        "api_key": TAVILY_API_KEY,
        "query": "What is the weather in Tokyo?"
    },
)

res.json()

{'query': 'What is the weather in Tokyo?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'Tokyo, Japan 14 day weather forecast - timeanddate.com',
   'url': 'https://www.timeanddate.com/weather/japan/tokyo/ext',
   'content': 'Tokyo 14 Day Extended Forecast. Weather Today Weather Hourly 14 Day Forecast Yesterday/Past Weather Climate (Averages) Currently: 68 °F. Partly sunny. (Weather station: Tokyo Heliport, Japan). See more current weather.',
   'score': 0.9930153,
   'raw_content': None},
  {'title': 'Hourly forecast for Tokyo, Japan - timeanddate.com',
   'url': 'https://www.timeanddate.com/weather/japan/tokyo/hourly',
   'content': 'Hour-by-Hour Forecast for Tokyo, Japan. Weather Today Weather Hourly 14 Day Forecast Yesterday/Past Weather Climate (Averages) Currently: 70 °F. Partly sunny. (Weather station: Tokyo Heliport, Japan). See more current weather.',
   'score': 0.9920312,
   'raw_content': None},
  {'title': 'Weather for Tokyo, Japan -

As you can see, we don't return much useful info here beyond that we have some URLs that *do contain* the information we need. Fortunately, we can extract this information via Tavily's `/extract` endpoint.

In [6]:
urls = [x["url"] for x in res.json()["results"]]

res = requests.post(
    f"{tavily_url}/extract",
    json={
        "api_key": TAVILY_API_KEY,
        "urls": urls
    },
)

In [7]:
res.json()

{'results': [{'url': 'https://www.timeanddate.com/weather/japan/tokyo',
   'raw_content': "Weather in Tokyo, Japan\nCool.\nFeels Like: 58\xa0°FForecast: 74 / 52\xa0°FWind: 2 mph ↑ from South\nUpcoming 5 hours\nSee more hour-by-hour weather\nForecast for the next 48 hours\n14 day forecast, day-by-dayHour-by-hour forecast for next week\nYesterday's weather\nCool. 72 / 58\xa0°FHumidity: 67%. Wind: 5 mph ↑ from North\nMore weather last week\nCurrently at nearby stations\nTokyo International Airport: (11\xa0mi)\nPassing clouds.\nYokota Ab: (20\xa0mi)\nClear. (1 hour ago)\nNational Airport: (39\xa0mi)\nPassing clouds.\nMore weather in Japan\nForecast for the next 2 weeks\nView historic weather\n74 / 52\xa0°F\n73 / 56\xa0°F\n58 / 54\xa0°F\n62 / 50\xa0°F\n62 / 48\xa0°F\n62 / 49\xa0°F\n63 / 52\xa0°F\n65 / 54\xa0°F\n69 / 59\xa0°F\n72 / 60\xa0°F\n72 / 60\xa0°F\n71 / 59\xa0°F\n69 / 59\xa0°F\n65 / 57\xa0°F\n64 / 56\xa0°F\nDetailed forecast for 14 days\n Need some help?\n© Time and Date AS 1995–2024

We can see that most requests didn't work, but that's okay we made multiple requests and fortunately received one result that looks perfect, we can extract that information like so:

In [9]:
print(res.json()["results"][0]["raw_content"])

Weather in Tokyo, Japan
Cool.
Feels Like: 58 °FForecast: 74 / 52 °FWind: 2 mph ↑ from South
Upcoming 5 hours
See more hour-by-hour weather
Forecast for the next 48 hours
14 day forecast, day-by-dayHour-by-hour forecast for next week
Yesterday's weather
Cool. 72 / 58 °FHumidity: 67%. Wind: 5 mph ↑ from North
More weather last week
Currently at nearby stations
Tokyo International Airport: (11 mi)
Passing clouds.
Yokota Ab: (20 mi)
Clear. (1 hour ago)
National Airport: (39 mi)
Passing clouds.
More weather in Japan
Forecast for the next 2 weeks
View historic weather
74 / 52 °F
73 / 56 °F
58 / 54 °F
62 / 50 °F
62 / 48 °F
62 / 49 °F
63 / 52 °F
65 / 54 °F
69 / 59 °F
72 / 60 °F
72 / 60 °F
71 / 59 °F
69 / 59 °F
65 / 57 °F
64 / 56 °F
Detailed forecast for 14 days
 Need some help?
© Time and Date AS 1995–2024

 © Time and Date AS 1995–2024. 
 Privacy & Terms




Okay so this is how we use the Tavily API to search the web, now let's implement this logic within a function which we can then use as a tool (ie action) for our ReAct agent.

In [16]:
def search(query: str):
    """Use this tool to search the web for information."""
    # first we need to search the web for the query
    res = requests.post(
        f"{tavily_url}/search",
        json={
            "api_key": TAVILY_API_KEY,
            "query": query
        },
    )
    # now get all the URLs from the search results
    urls = [x["url"] for x in res.json()["results"]]
    # now extract the information from the URLs
    res = requests.post(
        f"{tavily_url}/extract",
        json={
            "api_key": TAVILY_API_KEY,
            "urls": urls
        },
    )
    # we return just the top result as otherwise we overload our LLM
    return res.json()["results"][0]["raw_content"]

Let's test our function:

In [12]:
print(search(query="What is the weather in Tokyo?"))

Yahoo Weather
My Locations
Around the World
Tokyo
Japan
Mostly Sunny
Forecast
5 PM
6 PM
7 PM
8 PM
9 PM
10 PM
11 PM
12 AM
1 AM
2 AM
3 AM
4 AM
5 AM
6 AM
7 AM
8 AM
9 AM
10 AM
11 AM
12 PM
1 PM
2 PM
3 PM
4 PM
Clear with a high of 42 °F (5.6 °C) and a 49% chance of precipitation. Winds NW at 24 mph (38.6 kph).
Night - Clear with a 28% chance of precipitation. Winds variable at 7 to 25 mph (11.3 to 40.2 kph). The overnight low will be 34 °F (1.1 °C).
Sunny today with a high of 54 °F (12.2 °C) and a low of 32 °F (0 °C).
Mostly cloudy today with a high of 56 °F (13.3 °C) and a low of 40 °F (4.4 °C).
Rain today with a high of 52 °F (11.1 °C) and a low of 40 °F (4.4 °C). There is a 66% chance of precipitation.
Rain today with a high of 47 °F (8.3 °C) and a low of 41 °F (5 °C). There is a 77% chance of precipitation.
Showers today with a high of 48 °F (8.9 °C) and a low of 42 °F (5.6 °C). There is a 75% chance of precipitation.
Mostly cloudy today with a high of 54 °F (12.2 °C) and a low of 39 °F 

Great, now let's define two more tools...

We will create a simple "current date and time tool".

In [14]:
from datetime import datetime

def date():
    """Use this tool to get the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

And very importantly, we will define a tool that will be triggered when our LLM would like to provide it's final answer to the user.

In [15]:
def answer(text: str):
    """Use this tool to provide your final answer to the user."""
    return text

Now we generate an additional part to our `system_prompt` to explain which tools are available to the LLM.

In [25]:
import inspect

# we get the various parameters/description from each tool function
tools = [search, date, answer]
tool_descriptions = [
    {
        "name": tool.__name__,
        "description": str(inspect.getdoc(tool)),
        "args": {
            k: str(v).split(": ")[1] for k, v in inspect.signature(tool).parameters.items()
        }
    }
    for tool in tools
]
tool_descriptions

[{'name': 'search',
  'description': 'Use this tool to search the web for information.',
  'args': {'query': 'str'}},
 {'name': 'date',
  'description': 'Use this tool to get the current date and time.',
  'args': {}},
 {'name': 'answer',
  'description': 'Use this tool to provide your final answer to the user.',
  'args': {'text': 'str'}}]

Now let's parse these into text instructions that can be added to our `system_prompt`.

In [28]:
tool_instructions = (
    "You have access to the following tools ONLY, no other tools exist:\n\n"
    + "\n".join([str(x) for x in tool_descriptions])
)
print(tool_instructions)

You have access to the following tools ONLY, no other tools exist:

{'name': 'search', 'description': 'Use this tool to search the web for information.', 'args': {'query': 'str'}}
{'name': 'date', 'description': 'Use this tool to get the current date and time.', 'args': {}}
{'name': 'answer', 'description': 'Use this tool to provide your final answer to the user.', 'args': {'text': 'str'}}


Now let's try calling our LLM again with these additional instructions.

In [37]:
res = ollama.chat(
    model="llama3.2:3b-instruct-fp16",
    messages=[
        {"role": "system", "content": f"{system_prompt}\n\n{tool_instructions}"},
        {"role": "user", "content": "What is Ollama in the context of AI?"},
    ],
    format="json",
    options={
        "temperature": 0.0
    }
)

step = res["message"]["content"]
print(step)

{
  "thought": {
    "thoughts": "I need to find out what Ollama refers to in the context of AI"
  },
  "action": {
    "tool": "search",
    "args": {"query": "Ollama AI"}
  }
}

  
  

  
  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  

  




Perfect! Our LLM has correctly generated the query we need. We can now parse this and pass it into the `search` tool as specified by our LLM.

In [36]:
import json

tool_choice = json.loads(res["message"]["content"])["action"]["tool"]
args = json.loads(res["message"]["content"])["action"]["args"]

# we use a dictionary to map the tool name to the tool function
tool_selector = {x.__name__: x for x in tools}

# now we select the tool and call it with the arguments
observation = tool_selector[tool_choice](**args)
print(observation)

Ollama Explained: Transforming AI Accessibility and Language Processing
In the rapidly evolving landscape of artificial intelligence (AI), accessibility and innovation are paramount. Among the myriad platforms and tools emerging in this space, one name stands out: Ollama. But what exactly is Ollama, and why is it garnering attention in the AI community? This article delves into the intricacies of Ollama, its methodologies, its potential impact on AI applications, and what this could mean for the future of human-machine interaction.
Table of Content
Understanding Ollama
Ollama stands for (Omni-Layer Learning Language Acquisition Model), a novel approach to machine learning that promises to redefine how we perceive language acquisition and natural language processing.
At its core, Ollama is a groundbreaking platform that democratizes access to large language models (LLMs) by enabling users to run them locally on their machines. Developed with a vision to empower individuals and organizat

Nice! Within the ReAct framework we would then pass this information back to our LLM via a new `observation` variable. Let's try.

In [39]:
iteration = json.loads(step)
iteration["observation"] = observation
iteration_str = json.dumps(iteration, indent=2)
print(iteration_str)

{
  "thought": {
    "thoughts": "I need to find out what Ollama refers to in the context of AI"
  },
  "action": {
    "tool": "search",
    "args": {
      "query": "Ollama AI"
    }
  },
  "observation": "Ollama Explained: Transforming AI Accessibility and Language Processing\nIn the rapidly evolving landscape of artificial intelligence (AI), accessibility and innovation are paramount. Among the myriad platforms and tools emerging in this space, one name stands out: Ollama. But what exactly is Ollama, and why is it garnering attention in the AI community? This article delves into the intricacies of Ollama, its methodologies, its potential impact on AI applications, and what this could mean for the future of human-machine interaction.\nTable of Content\nUnderstanding Ollama\nOllama stands for (Omni-Layer Learning Language Acquisition Model), a novel approach to machine learning that promises to redefine how we perceive language acquisition and natural language processing.\nAt its cor

Now we feed this back into our chat.

In [44]:
res = ollama.chat(
    model="llama3.2:3b-instruct-fp16",
    messages=[
        {
            "role": "system",
            "content": f"{system_prompt}\n\n{tool_instructions}"
        },
        {"role": "user", "content": "What is Ollama in the context of AI?"},
        {"role": "assistant", "content": f"Step 1:\n{iteration_str}\n\nWhat do I do next to answer the user's question..."},
    ],
    format="json",
    options={
        "temperature": 0.0
    }
)

step2 = res["message"]["content"]
print(step2)

{ "action": { "tool": "answer", "args": {"text": "Ollama is a novel approach to machine learning that promises to redefine how we perceive language acquisition and natural language processing."}} }


There we go! Our final answer from the LLM is:

In [45]:
step2_json = json.loads(step2)
print(step2_json["action"]["args"]["text"])

Ollama is a novel approach to machine learning that promises to redefine how we perceive language acquisition and natural language processing.


Perfect, now let's take everything we've done so far and use it to construct our ReAct agent.

In [ ]:
from typing import Callable


class ReActAgent:
    def __init__(self, tools: list[Callable]):
        self.messages = []
        self.tools = {x.__name__: x for x in tools}
        tool_descriptions = [
            {
                "name": tool.__name__,
                "description": str(inspect.getdoc(tool)),
                "args": {
                    k: str(v).split(": ")[1] for k, v in inspect.signature(tool).parameters.items()
                }
            }
            for tool in tools
        ]
        tool_instructions = (
            "You have access to the following tools ONLY, no other tools exist:\n\n"
            + "\n".join([str(x) for x in tool_descriptions])
        )
        self.system_prompt = f"{system_prompt}\n\n{tool_instructions}"

    def __call__(self, prompt: str, max_steps: int = 10):
        step_count = 1
        while step_count < max_steps:
            self.messages.append({"role": "user", "content": prompt})


        if step_count == max_steps:
            # force final answer


    def _call_llm(self, messages: list[dict]):
        res = ollama.chat(
            model="llama3.2:3b-instruct-fp16",
            messages=self.messages,
            format="json",
        )
        return res["message"]["content"]

    def _parse_step(self, step: str):
        step_dict = json.loads(step)
        tool_choice = step_dict["action"]["tool"]
        args = step_dict["action"]["args"]
        tool_selector = {x.__name__: x for x in self.tools}
        observation = tool_selector[tool_choice](**args)
        return observation

    def run_step(self, step_number: int, scratchpad: str):
